# Gate Set Tomography

In [1]:
import numpy as np
import qiskit

# For GST experiment and followed analysis
from qiskit_experiment.library import GateSetTomography
from qiskit_experiment.library.gateset_tomography.gatesetbasis import default_gateset_basis, gram_matrix_rank

from qiskit.extensions import XGate, YGate, HGate, U2Gate, IGate, SXGate, RZGate
from qiskit.quantum_info import Operator, Choi, PTM, DensityMatrix

# Noisy simulator backends
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeParis
from qiskit.providers.aer.noise import NoiseModel
from qiskit import Aer

backend_fake_P = AerSimulator.from_backend(FakeParis())

In this tutorial, we introduce how we can implement a GST experiment in Qiskit, run it on various backends and read the analysis results that can be performed using two fitters in addition to the accuracy measures of the final results of the experiment.  

## Overview

### Mathematical preliminaries

This tutourial will depend on the following mathematical concepts from the the quantum computing field:

* A quantum system state of $N$ qubits is described by a **density matrix**, $\rho$, which is a $d\times d$ matrix with $d=2^N$. A valid density matrix $\rho$ should satisfy the following two physical constraints:<br> 
1. $Tr(\rho)=1$
2. $\rho$ is positive semi-definite (PSD), since it is also Hermitian $(\rho=\rho^\dagger)$ this means it has only nonnegative values.<br><br>

* A **general measurement** of the quantum system state is called POVM, where POVM stands for **P**ositive **O**perator **V**alued **M**easurement and is completely described by a set $\left\{ E_m=M_m^\dagger M_m \right\} _m$ where $$\sum_m E_m =\sum_m  M_m^\dagger M_m =\mathcal{I}$$ and $E_m\geq 0$. <br>The probability to obtain an outcome $m$ (e.g. $m$ is measuring the qubit in state $|0\rangle$ for $E_0 = |0\rangle \langle 0|$) is:$$p_m=Tr(\rho E_m)=\langle\langle E_m|\rho \rangle\rangle$$where $|E_m\rangle\rangle$ and $|\rho\rangle\rangle$ are **superstates**. This representation is called the "superoperator" formalism, in which operators in the Hilbert space of dimension $d\times d$ are represented as "vectors" or states in a larger space, called **Hilbert-Schmidt space** of dimention $d^2\times d^2$. For this tutourial, we'll regard $|E_m\rangle\rangle$ and $|\rho\rangle\rangle$ as superstates expressed in a "vector form" in the normalized Pauli basis that spans the Hilbert-Schmidt space (PTM representation). For example, $$E_0 =|0\rangle \langle 0|=\begin{pmatrix}
1 & 0 \\
0 & 0 
\end{pmatrix} = \frac{1}{\sqrt{2}} \left( P_0 +P_3 \right) \rightarrow |E_0\rangle\rangle =\frac{1}{\sqrt{2}}(1,0,0,1)^T$$<br> where $P_0$ and $P_3$ are the single qubit noramalized Paulis $\frac{1}{\sqrt{2}} I$ and $\frac{1}{\sqrt{2}} Z$ respectively.<br><br>


* A **quantum channel**, $G$, is a linear map that transforms a quantum state $\rho$ to $G(\rho)$ and it satisfies the following two physical constraints:
 1. Trace preservation (TP) - TP means that $Tr(G(\rho))$ is presereved and is equal to 1.
 2. Complete Positivity (CP) - If $G$ keeps $\rho$ positive, then this is a positive map. Complete positivity is a stronger condition, according to which when $G\times \mathcal{I}_A$ acts on a larger system $\rho\otimes \rho_A$, it preserves positivity.

 If $G$ satisfies both of the conditions, we say it is a **CPTP** map.<br><br>
 Quantum channels may be displayed using various representations. For this tutourial, we will be focusing on the following two:
<br> 
 - Pauli transfer matrix (PTM) representation- Here, quantum channels are displayed in the normalized Pauli basis, where the $\{i,j\}$ of the PTM matrix of a quantum channel, $G$, is given by: $$(R_G)_{i,j} =Tr(P_i G(P_j)), $$ where $\{P_i\}_{i=1}^d$ is the orthonormal Pauli strings basis and $d=(2^{2N})$ is the Hilbert Schmidth space dimention where $N$ is the number of qubits.<br> For more information on PTM implementation in Qiskit, see [PTM CLASS](https://qiskit.org/documentation/stubs/qiskit.quantum_info.PTM.html).The TP condition on $G$ is equivalent to $(R_G)_{0j}=\delta_{0j}$.<br>

 - Choi-Jamiolkowski (CJ) matrix representation- This representation is based on the Choi-Jamiolkowski isomorphism according to which, every quantum channel can be associated with a CJ matrix. CJ matrix, is a map from the PTM matrix $R_G$ to a density matrix in a larger Hilbert Space  that is defined as $$\rho_G = \frac{1}{2^N}\sum_{i,j=1}^{d^2} (R_G)_{i,j} P_j^T\otimes P_i.$$ <br>For more information about Choi representation implementation in Qiskit, see [Choi CLASS](https://qiskit.org/documentation/stubs/qiskit.quantum_info.Choi.html). Note that the implementation in Qiskit, returns an object which is equal to the density matrix up to a factor of $2^N$. CP condition pn $G$ is equivalent to $\rho_G$ is PSD. 
<br>


***


### Gate Set Tomography Experiment
Gate-Set Tomography is used to perform **full characterization** of quantum processes using measurement data we obtain from measuring a specific set of quantum circuits.

Gate set tomography addresses one of the main shortcomings of quantum process tomography: the state preparation and measurements **(SPAM)** errors. GST deals with the SPAM errors self-consistently by including the gates used for both initializing and measuring the qubits in the gate set which GST is trying to reconstruct from processing measurement data.

<br/>

To do this, GST performs full characterization of the so-called **gate set** which includes:
- A set of gates: $\mathcal{G} = (G_0, ..., G_M)$, 
- The native state: $|\rho\rangle\rangle$ 
- The native measurement: $|E\rangle\rangle$,

where:

$|\rho\rangle\rangle$ and $|E\rangle\rangle$ are a native preparation state and measurement of the quantum device. The $|\cdot \rangle\rangle$ here as explained in the above section indicates they are displayed the PTM representation; as superstates in the normalized Pauli strings basis $\{|i\rangle\rangle \ | \ i\in [0,1,2,...,d^2-1]\}$- which refers to the set $\{P_i\}_{i=0}^{d^2-1}$. 

$\mathcal{G}$ should include the gates we wish to characterize, and additional gates such that 
from the set of gates, $[G_i]_{i=0}^{M}$, we are able to construct the SPAM gates, $\mathcal{F}=[F_i]_{i=0}^{d^2-1}$, which when acting on the native state and measurement it creates:
- An informationally complete set of initial states: $|\rho_i\rangle\rangle= R_{F_i}|\rho\rangle$, $i=0,1,2,..,d^2-1$

and

- An informationally complete set of measurements: $|E_i\rangle\rangle= R_{F_i}|E\rangle$, $i=0,1,2,..,d^2-1$.

** $d^2$ again is the superspace dim.= $2^{2\cdot N}$ and $R$ with subscript ${F_i}$ or ${G_k}$ is the PTM representation (expressed in terms of the Pauli strings basis) of $F_i$ and $G_k$ respictively..

Each $F_i$ is constructed from a sequence of the gates in the gate set. For example, we can have
$F_1=G_5$ and $F_2=G_1o\ G_3$ ($G3$ followed by $G1$).

<br/>

To be able to characterize the gate set, GST uses the following **three sets of circuits**: 

1. Circuits that measure elements of the form: $\langle\langle E|R_{F_i} R_{G_k} R_{F_j}|\rho \rangle\rangle$

![alt text](GST_circuit_rho_F_G_F_E_element1.png "Title")

Note that, for each $k$, we have $d^4$ different circuits. If $R_{F_i}$'s were known, then this will be enough data to reconstruct $R_{G_k}$ which has $d^4$ unknown entries as well. However, since $R_{F_i}$ and $R_{F_j}$ are assumed to be unknown, we need more circuits to be able to reconstruct the gates. 

2. Circuits that measure elements of the form: $\langle\langle E|R_{F_i}R_{F_j}|\rho\rangle\rangle$ (the Gram Matrix -g- Elements)

![alt text](gram_matrix_elements.png "Title")

3. Circuits that measure elements of the form: $\langle\langle E|R_{F_i}|\rho\rangle\rangle$ 

![alt text](GST_circuit_rho_F_E_element.png "Title")

** Taking the first gate of $\mathcal{G}$ and $\mathcal{F}$ as the identity (null) gate, makes the second and third sets just two special cases of the first set. 

<br/>

### Analyzing the GST experimental data

The obtained experimental data, from the above three sets of circuits, is enough to fully characterize the gate set. To do this, we may use two fitters:

**Linear inversion**

As in quantum process tomography, we use the experimental measurements $p_{ijk}=\langle\langle E_i|R_{G_k}|\rho_j\rangle\rangle=(p_k)_{ij}$ to reconstruct $G_k$. However, as explained above, since the initial states and measurements are unknown, there is a need for performing more measurements to gain further information. For this purpose, $\langle\langle E|R_{F_i}R_{F_j}|\rho\rangle\rangle$ and $\langle\langle E|R_{F_i}|\rho\rangle\rangle$ are being measured.

Having all this experimental data, we note that, firstly, $(p_k)_{ij}$ can be rewritten as:
$$(p_k)_{ij}=(AR_{G_k}B)_{ij},$$
where the matrices $A$ and $B$ are given by
$$A=\sum_i |i\rangle\rangle \langle\langle E|R_{F_i}$$
$$B=\sum_i R_{F_j}|\rho\rangle\rangle \langle\langle j|,$$

Secondly, the second set of measurements corresponding to having $G_k=${}, yields:
$$\langle\langle E|R_{F_i}R_{F_j}|\rho\rangle\rangle = (AB)_{ij}=g_{ij}$$ 

Hence, this set of measurements provides us with more information to reconstruct the gates $G_k$'s. However, We still can not reconstuct $B$ or $A$ separately.
Now, if we multiply the expression for $p_k$ from the left side by the inverse of the gram matrix ($g$), we find
$$g^{-1}p_k=B^{-1} R_{G_k} B\equiv \tilde{R}_{G_k}$$

In a similar way, $\langle\langle E|R_{F_i}|\rho\rangle\rangle$ gives us:
$$g^{-1} \sum_i |i\rangle\rangle \langle\langle E|R_{F_i}|\rho\rangle\rangle = B^{-1}|\rho\rangle\rangle \equiv |\tilde{\rho}\rangle\rangle$$
and $$\sum_i \langle\langle E|R_{F_i}|\rho\rangle\rangle \langle\langle i| = \langle\langle E|B \equiv \langle\langle \tilde{E}|$$

Note that, the measurements of any two sets of $\left\{ |\tilde{\rho}\rangle\rangle,\ \langle\langle \tilde{E}|,\ \tilde{R}_{G_k} \right\}$, that are related by a gauge transformation $B$ are equivalent as they always lead the same expectation values - all the $B$'s will be canceled. In particular, they are equivalent to the case where $B$ is the identity, i.e., the original gates, measurement and state.
 
However, as $B$ is immeasurable, it can be found using optimization methods such that the final results with the optimized gauge are close as much as possible to some target set. In qiskit GST code, the target is chosen to be the ideal set and the $B$ matrix is found by minimizing the RMS discrepancy of the two sets as defined in section 3.4.4 in [1]. 
**Drawbacks:** 
1. The gauge problem. 
2. This method does not gaurantee the results are TP and CP. 

**Maximum likelihood estimation** 
    As in every maximum likelihood problem, we have a set of observations, which are the measurement results of the circuits, $m_{ijk}$ for each one of the circuits $\{i,j,k\}$ and they are equal to $\frac{\#\ of\ counts_{|00\cdots0\rangle}}{shots}$. In addition, we assume a probabilistic model which is parametrized that describes the experiment results, and the maximum liklihood solution is given as the parametrization for which the observed results data under the assumed probabilistic model and the physical constraints are the most probable. Given a parametrization: $\langle\langle E(\vec{t})|$, $|\rho(\vec{t}\rangle\rangle$ and $R_{G_k}(\vec{t})$'s, the parametrized probabilitity to measure $|E\rangle\rangle$ in circuit $\{i,j,k\}$ is
$$p_{ijk}=\langle\langle E(\vec{t})|R_{G_k}(\vec{t})|\rho(\vec{t}\rangle\rangle.$$
Hence, assuming a multinomial probabilistic model for the GST experiment, a possible choice for the liklihood function is 
$$\mathcal{L}(\mathcal{G})=\Pi_{ijk} p_{ijk}^{m_{ijk}} (1-p_{ijk})^{1-m_{ijk}}.$$
Which is exactly equal to the multinomial probability distribution function up to the constant binomial coefficients and after exponentiating the whole probility distribution function by $\frac{1}{shots}$.

A simpler form can be obtained by invoking the central limit theorem (for further information, see section 3.5 in [1])). Taking the negative log of the central limit theorem results leaves us with the weighted least squares cost function, which we need to minimize:
$$\mathcal{L}_{WLS} =\sum_{ijk} \left( p_{ijk}-m_{ijk} \right) ^2 /{\sigma_{ijk}}^2$$
where $\sigma_{ijk}$ are the sampling errors of each circuit measurement.


The form of the cost function Qiskit GST is even simpler than the WLS form and is equal to simply least square cost function given as 
$$ \mathcal{L}_{LS} =\sum_{ijk} \left( p_{ijk}-m_{ijk} \right)^2$$

The MLE solution will be given as the parametrization $\vec{t}$ that minimizes $\mathcal{L}_{LS}$ and at the same time, respects the physical constraints: The quantum gates are CPTP, $rho$ is a valid density matrix and hence it is PSD and with trace $1$, and finally, $E$ is a POVM and hence $E$ and $\mathcal{I}-E$ are positive.

Here the default is to use the solution obtained by linear inversion after the gauge optimization as a starting point for the optimization problem (the user can choose not to and provide his/her own starting point). It is important to provide a starting point as the cost function is highly nonlinear and therefore a good starting point will help the optimization algorithm to converge to the correct minima. **Advantage:** The results satisfy the physical constraints and does not have a gauge problem.


***

# GST Experiment

To run GST experiment, we need to provide the following parameters:

1) `qubits`: A list of physical qubits GST is performed on.

2) `basis_gates`: A dictionary describing a set of gates $\mathcal{G}$, GST experiment characterizes.

3) `spam_gates`: A dictionary describing the decomposition of the SPAM gates in terms of the `basis_gates`.

4) `additional_gates`: A list of gates to be added to the gateset to be characterized.

5) `only_basis_gates`: A boolean variable that indicates whether the user provided only basis gates from which the algorithm needs to construct a possible set of SPAM gates.


## Gate set data arguments

The gate set data we pass in to the GST experiment can be any of the following:
    
a) Two dictionaries describing both of the basis gates, i.e., $\mathcal{G}$, and one for the spam gates, $\mathcal{F}$, where:<br> `
 - `basis_gates`: the dictionary keys should include the gates labels, and the corresponding values are functions describing the action of the gate on the qubits that take QuantumCircuit and QuantumRegister as arguments.
 - `spam_gates`: The dictionary of the `spam_gates` keys are the indices of each spam gate that starts with the letter F, followed by the index value: 'F0', 'F1',...etc. The values of the `spam_gates` are tuples that describe the decomposition in terms of the `basis_gates`, where each tuple includes the labels in the decomposition in a specific order.
<br>

In this case, `only_basis_gates_1 ` is False, which is also the default value.<br>
<br> The parameter `additional_gates` includes the gates we would like to add to the gate set to characterize, this is provided as a dictionary as well, where the keys are the names and the values can be either a qiskit gate object (e.g. `Gate`) or a function as in the `basis_gates`. 

Below, you may see an example for the single-qubit case:

In [2]:
# Define the basis gates as a dictionary, where the keys are the names of the gates, and the
#values are the corresponding gates of type: FunctionType.
basis_gates_1 = {
    'Id': lambda circ, qubit: None,
    'X_Rot_90': lambda circ, qubit: circ.append(U2Gate(-np.pi / 2, np.pi / 2), [qubit]),
    'Y_Rot_90': lambda circ, qubit: circ.append(U2Gate(0, 0), [qubit])
    }

# Define the SPAM gates which are constructed from the basis gates and give an informationally
#complete set of initial states and measurements when applied on rho=|0><0| and E=|0><0|. 
#The spam dictionary should be of the following form:

spam_gates_1 = {
    'F0': ('Id',),
    'F1': ('X_Rot_90',),
    'F2': ('Y_Rot_90',),
    'F3': ('X_Rot_90', 'X_Rot_90')
}

# The default of only_basis_gates is False, so we don't need to pass it in in this case.
only_basis_gates_1 = False

#Additional_gates can be none as well
additional_gates_1 = None

- **Note:** For the sake of obtaining good tomography results, it is important to check before running the experiment that the gram matrix has a full rank and is not singular or ill-conditioned, so it can be 
inverted appropriately. In addition, the gate set should be picked so the gram matrix singular values are
as large as possible. To display the gram matrix singular values and its rank, we may use the gram_matrix_rank function, that takes three arguments:
1. `num_qubits`- Number of qubits GST is performed on.
2. `spam_gates_labels`- A list of spam gates labels.
3. `basis_gates` - Same argument as `basis_gates` provided to GST experiment.
<br>
This function returns both the gram matrix rank and its singular values.

In [3]:
gram_rank, singular_values = gram_matrix_rank(num_qubits = 1, spam_gates_labels = list(spam_gates_1.values()) ,basis_gates = basis_gates_1)
print('Gram matrix rank:', gram_rank)
print('Gram matrix singular values:', singular_values)

Gram matrix rank: 4
Gram matrix singular values: [1.78077641 1.         0.5        0.28077641]


<br/>

b) If `basis_gates` is 'default' or None, the Qiskit built-in default gate sets for $\mathcal{G}$ and $\mathcal{F}$ will be used. The default is only available for single qubit or two qubits, and both are built from the backends basis gates. Additional gates GST should characterize again are provided as a dictionary: `additional_gates`.<br><br>
     
### For the single-qubit case
The gateset basis of the default is: <br>
`G = {'I', 'SX','RZ_pi/2', 'RZ_-pi/2'}`

and the SPAM gates are:

```
{
'F0': ('Id',)
'F1': ('SX',),
'F2': ('RZ_pi/2','SX','RZ_-pi/2',),
'F3': ('SX', 'SX')
}
```

Where `Id` is the identity gate `IGate()`, `SX` is `SXGate()`,` RZ_pi/2` is `RZGate`($\theta=\pi/2$) and `RZ_-pi/2` is `RZGate`($\theta=-\pi/2$).

### For the two-qubits case:

`G={'I I','X I', 'I X', 'RZ_pi/3 I', 'I RZ_pi/3', 'I SX', 'SX I', 'CX' )}`

And the SPAM gates are:
```
{
'F0': ('I I',)
'F1': ('X I',)
'F2': ('I X',)
'F3': ('X I', 'I X',)
'F4': ('I X', 'SX I', 'CX',)
'F5': ('I SX', 'I RZ_pi/3', 'I SX',)
'F6': ('SX I', 'RZ_pi/3 I', 'SX I',)
'F7': ('X I', 'I SX', 'I RZ_pi/3', 'I SX')
'F8': ('I X', 'I SX', 'CX', 'I SX')
'F9': ('I X', 'SX I', 'RZ I', 'SX I')
'F10': ('RZ_pi/3 I', 'RZ_pi/3 I', 'RZ_pi/3 I', 'SX I')
'F11': ('I RZ_pi/3', 'I RZ_pi/3', 'I RZ_pi/3', 'I SX')
'F12': ('I SX', 'SX I', 'CX', 'I SX')
'F13': ('X I', 'I RZ_pi/3', 'I RZ_pi/3', 'I RZ_pi/3', 'I SX')
'F14': ('I SX', 'I RZ_pi/3', 'SX I', 'CX', 'I SX')
'F15': ('RZ_pi/3 I', 'RZ_pi/3 I', 'RZ_pi/3 I', 'CX', 'I SX', 'CX')
}
```
where here, 'A B' stands for applying a gate B on the first qubit, and gate A on the second qubit. `'RZ_pi/3'` is the `RZGate`$(\theta=\pi/3)$.


In [4]:
basis_gates_2 = 'default'
#if basis_gates are default, the spam gates are chosen to be the default spam gates automatically.
spam_gates_2 = 'default'

# Alternatively,
# basis_gates_2 = None

only_basis_gates_2 = False
additional_gates_2 = {'h': HGate()}

c) Providing only a set of basis gates the gate set is composed of. In this case, the boolean variable
`only_basis_gates` should take the True value. 
For the spam gates construction, a built-in function, called:
gatesetbasis_constrction() from 'qiskit_experiment/library/gateset_tomography/gatesetbasis.py' will be used to construct a set of spam gates from the provided basis gates which are informationally complete and correspond to a full rank gram matrix.

For example, the provided gate set data can simply be:

In [5]:
basis_gates_3 = {
 'Id': lambda circ, qubit: None,
 'H': lambda circ, qubit: circ.append(HGate(), [qubit]),
 'Y': lambda circ, qubit: circ.append(YGate(), [qubit]),
 'X_Rot_90': lambda circ, qubit: circ.append(U2Gate(-np.pi / 2, np.pi / 2), [qubit]),
}
spam_gates_3 = None
only_basis_gates_3 = True
additional_gates_3 = None

<br/>

## Running gate set tomography on 1-qubit

In [6]:
#Creating a GateSetTomography experiment instance:

#First option: Arbitrary basis gates set and spam gates set provided by the
#user as two dictionaries-
"""           
gstexp1 = GateSetTomography(qubits=[0],
                            basis_gates=basis_gates_1,
                            spam_gates=spam_gates_1,
                            additional_gates=additional_gates_1 ,
                            only_basis_gates=only_basis_gates_1)
"""
################################
#Second option: Default gate sets, with additional gates-

gstexp1 = GateSetTomography(qubits=[0],
                            basis_gates=basis_gates_2,
                            spam_gates=spam_gates_2,
                            additional_gates=additional_gates_2,
                            only_basis_gates=only_basis_gates_2)
"""
################################
#Third option:Providing only basis gates without SPAM-

gstexp1 = GateSetTomography(qubits=[0],
                            basis_gates=basis_gates_3,
                            spam_gates=spam_gates_3,
                            additional_gates=additional_gates_3 ,
                            only_basis_gates=only_basis_gates_3)

################################
"""
#Run GST experiment
gstdata1 = gstexp1.run(backend_fake_P).block_for_results()


In [7]:
#To view and draw the GST experiment circuits, we may use the method `circuits` of
#the `GateSetTomography` class which returns a list of all the GST circuits. 

#For instance, to draw the 4th, and 10th GST circuit:

gstexp1.circuits(backend = backend_fake_P)[4].draw()


┌────┐ ░  ░ ┌─┐
q_0: ┤ √X ├─░──░─┤M├
     └────┘ ░  ░ └╥┘
c: 1/═════════════╩═
                  0

In [8]:
gstexp1.circuits(backend = backend_fake_P)[10].draw()

┌─────────┐┌────┐┌──────────┐ ░  ░ ┌─────────┐┌────┐┌──────────┐┌─┐
q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(-π/2) ├─░──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(-π/2) ├┤M├
     └─────────┘└────┘└──────────┘ ░  ░ └─────────┘└────┘└──────────┘└╥┘
c: 1/═════════════════════════════════════════════════════════════════╩═
                                                                      0

In [9]:
#View final analysis results
for result in gstdata1.analysis_results():
    print(result)

DbAnalysisResultV1
- name: GST Experiment properties
- value: {'fitter': 'scipy_optimizer_MLE_gst', 'fitter_time': 7.833991050720215, 'fitter_initial_guess': 'linear_inversion', 'GST gates': ['Id', 'SX', 'RZ_pi/2', 'RZ_-pi/2', 'h'], 'GST SPAM gates': {'F0': ('Id',), 'F1': ('SX',), 'F2': ('RZ_pi/2', 'SX', 'RZ_-pi/2'), 'F3': ('SX', 'SX')}}
- device_components: ['Q0']
- verified: False
DbAnalysisResultV1
- name: gst estimation of meas
- value: Operator([[ 0.9861288 +0.j        , -0.01690239+0.01690239j],
          [-0.01690239-0.01690239j,  0.01161372+0.j        ]],
         input_dims=(2,), output_dims=(2,))
- device_components: ['Q0']
- verified: False
DbAnalysisResultV1
- name: gst estimation of rho
- value: DensityMatrix([[ 9.96045345e-01+0.j        , -6.34172893e-04+0.00063417j],
               [-6.34172893e-04-0.00063417j,  3.77612773e-03+0.j        ]],
              dims=(2,))
- device_components: ['Q0']
- verified: False
DbAnalysisResultV1
- name: gst estimation of Id
- value: Cho

### Tomography Results

The main results of GST experiment, are the fitted native state and measurement and all the fitted gates in the gate set. The naive state is displayed as a density matrix, the measurement in an opertaor form and all the gates in the Choi representation.

In [10]:
native_state_result = gstdata1.analysis_results("gst estimation of rho")
print(native_state_result.value)

DensityMatrix([[ 9.96045345e-01+0.j        , -6.34172893e-04+0.00063417j],
               [-6.34172893e-04-0.00063417j,  3.77612773e-03+0.j        ]],
              dims=(2,))


In [11]:
native_measurement_result = gstdata1.analysis_results("gst estimation of meas")
print(native_measurement_result.value)

Operator([[ 0.9861288 +0.j        , -0.01690239+0.01690239j],
          [-0.01690239-0.01690239j,  0.01161372+0.j        ]],
         input_dims=(2,), output_dims=(2,))


In [12]:
#The quantum processes results are displayed in the Choi representation.
SX_gate_result = gstdata1.analysis_results("gst estimation of SX")
print('SX_GST_Choi:', SX_gate_result.value)

#To display the result in a PTM representation:
print('SX_GST_PTM:', PTM(SX_gate_result.value))

#To return the data of a Choi or PTM objects, we print the data attribute:

print('SX_GST_Choi_data:', SX_gate_result.value.data)


SX_GST_Choi: Choi([[ 0.52394663+0.j        , -0.04539346+0.48517271j,
       -0.04482888+0.4732512j ,  0.49723592+0.0735526j ],
      [-0.04539346-0.48517271j,  0.49314756+0.j        ,
        0.4756896 -0.01078227j,  0.00535762-0.50043023j],
      [-0.04482888-0.4732512j ,  0.4756896 +0.01078227j,
        0.46271138+0.j        ,  0.01686043-0.48923755j],
      [ 0.49723592-0.0735526j ,  0.00535762+0.50043023j,
        0.01686043+0.48923755j,  0.52019443+0.j        ]],
     input_dims=(2,), output_dims=(2,))
SX_GST_PTM: PTM([[ 1.        +0.j, -0.03947126+0.j, -0.02717904+0.j,  0.01709418+0.j],
     [-0.02853303+0.j,  0.97292551+0.j,  0.06277033+0.j, -0.06225388+0.j],
     [ 0.00406484+0.j, -0.08433488+0.j,  0.02154632+0.j, -0.97441026+0.j],
     [-0.01334199+0.j, -0.0501865 +0.j,  0.97368143+0.j,  0.04414106+0.j]],
    input_dims=(2,), output_dims=(2,))
SX_GST_Choi_data: [[ 0.52394663+0.j         -0.04539346+0.48517271j -0.04482888+0.4732512j
   0.49723592+0.0735526j ]
 [-0.04539346-0.

### Additional state metadata
Additional data about the fidelity is stored in the extra data of the AnalysisResult of each gate.
The fidelity measure, computes the avarage gate fidelity of GST results and some target using the function: 'qiskit.quantum_info.average_gate_fidelity'. If the target and the GST result are both non unitary (which is usually the case when the target is noisy), the 'qiskit.quantum_info.process_fidelity' measure will be used.

The target's default, as in the above experiment, is the ideal gates. However, the user can provide 
a custom target set as in the example shown in the next section.

To show the fidelity of the gates with the default target of the previous example, we type:

In [13]:
print('extra:\n', gstdata1.analysis_results("gst estimation of SX").extra)

extra:
 {'Average gate fidelity': 0.9868361993961127}


In [14]:
print('extra:\n', gstdata1.analysis_results("gst estimation of RZ_pi/2").extra)

extra:
 {'Average gate fidelity': 0.9754095826563655}


### Comparing the GST results to an arbitrary target set

As a default, the Qiskit GST algorithm compares the final experiment to the ideal set of gates. However, the user can provide an arbitrary target set to compare the results to.
The target set is a dictionary, where the keys are the labels of the gates,
and the corresponding values are provided as `Operator`, `Gate`, `FunctionType`, `PTM` or `Choi`. 
The `target_set` parameter is passed in as an analysis option.

For example:

In [15]:
basis_gates = basis_gates_3

#Noise model: Amplitude damping applied on each qubit after each gate. Th PTM representation of
#amplitude damping channel: 
gamma = 0.05
noise_ptm_AD = PTM(np.array([[1, 0, 0, 0],
                             [0, np.sqrt(1 - gamma), 0, 0],
                             [0, 0, np.sqrt(1 - gamma), 0],
                             [gamma, 0, 0, 1 - gamma]]))
         
# We can add targets E and rho, but they are not relevant, as the fidelity measure is applied only 
#on the target set.
target_set = {
 'Id':Choi(IGate()),
 'H': Choi(HGate()),
 'Y': Choi(YGate()),
 'X_Rot_90': Choi(U2Gate(-np.pi / 2, np.pi / 2)),
}
         
#As the gates after the noise in PTM representation is simply the multiplication of the noise
#channel by each gate channel, the noisy target set can be obtained in the PTM representation via:
target_set_noisy = {}
for key in target_set:
    if key != 'I':
        target_set_noisy[key]= PTM(np.dot(noise_ptm_AD, PTM(target_set[key]).data))
    
#Noise model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(noise_ptm_AD, ['sx', 'x', 'rz'])
backend_aer = Aer.get_backend("aer_simulator")

#GST experiment:
gstexp4=GateSetTomography(qubits=[0], basis_gates=basis_gates_3,
                        only_basis_gates=True)
gstexp4.set_analysis_options(target_set = target_set_noisy)

gstdata4 = gstexp4.run(backend=backend_aer, noise_model=noise_model).block_for_results()

additional_gate_result = gstdata4.analysis_results("gst estimation of Y")
print('GST estimation of Y:\n', additional_gate_result.value)
print('extra:\n', additional_gate_result.extra)



Input channel is not TP. Tr_2[Choi] - I has non-zero eigenvalues: [-0.01557181  0.01557181]
Input channel is not TP. Tr_2[Choi] - I has non-zero eigenvalues: [-0.09479954  0.09479954]
Input channel is not TP. Tr_2[Choi] - I has non-zero eigenvalues: [-0.011291  0.011291]
Input channel is not TP. Tr_2[Choi] - I has non-zero eigenvalues: [-0.08526228  0.08526228]


GST estimation of Y:
 Choi([[ 0.01114289+0.j        ,  0.03658832-0.00360486j,
       -0.02617626-0.00396122j,  0.00221652+0.00395273j],
      [ 0.03658832+0.00360486j,  0.99137079+0.j        ,
       -0.9852594 -0.03006283j,  0.02624565+0.01496864j],
      [-0.02617626+0.00396122j, -0.9852594 +0.03006283j,
        0.99517027+0.j        , -0.0275813 -0.00961493j],
      [ 0.00221652-0.00395273j,  0.02624565-0.01496864j,
       -0.0275813 +0.00961493j,  0.00231606+0.j        ]],
     input_dims=(2,), output_dims=(2,))
extra:
 {'Process fidelity': 0.9866632740055256}


### Tomography Fitters and additional analysis options:

There are two fitters that can be used for the GST experiment analysis, which can be set as analysis options, by setting `fitter = 'name of the fitter'`.

$\bf{1.}$ The first fitter is the **linear inversion fitter**: 'linear_inversion_gst'.As there is a gauge freedom in this case, the gauge is found using scipy optimization method, which searched for the similarity matrix $B$ that minimizes the RMS distance between the ideal set and the linear inversion results with the gauge $B$. 

The rsults of this fitter as mentioned earlier are not physical as well. Namely,
they are not TP or CP. The user however can choose to rescale the results to be TP or not CP or both. To do this, the user should set the analysis option `rescale_tp` and `rescale_cp` to be true. The default value of `rescale_tp` and `rescale_cp` is False for both. The values of both will be displayed in
the 'GST Experiment properties' AnalysisResult.

$\bf{2.}$ The second fitter which is the default fitter is the **MLE fitter**: 'scipy_optimizer_MLE_gst'. The results of this fitter are always CPTP and hence, `rescale_tp` and `rescale_cp` options are not relevant here. 

For this fitter, a `fitter_initial_guess` can be passed in as an analysis option. If the `fitter_initial_guess` is None, the optimization will be performed without an initial point and it may fail. It also takes as a string either "default" or "linear_inversion" and in both cases it uses linear inversion result as a starting solution. 

The user can provide an arbitrary initial guess for the MLE fitter which is a dictionary, including the names of the state, measurements and gates and their corresponding values, where:

- The value of rho, whose key is 'rho', can be a DensityMatrix or an array representing the PTM representation in the orthonormal
Pauli basis composed of strings of the form: $$\Pi_{i_0 i_1 \cdots i_N} P_{i_0}\otimes P_{i_1}\otimes \cdots \otimes P_{i_{N-1}}$$, where $P_{i_j}\in \frac{1}{\sqrt{2}}[I, X, Y, Z]$ and 
$N$ is the number of qubits. 
- The value of E, whose key should be 'meas' is an array representing the `PTM` representation of E. 
- The value of each one of the gates can be of type: `FunctionType`, `Choi`, `PTM`, `Gate` or `Operator`.

**Notes**: 
 1. The dictionary defined for the `fitter_initial_guess` should start with the data for "rho", then those for "meas" (E) and finally the data for each one of the gates in the gates.
 2. In the examples above, the default fitter which is the MLE fitter, with the default fitter initial guess which is the linear inversion results, were used.

#### Example for displaying the rescaled linear inversion GST results:

In [16]:
gstexp5 = GateSetTomography(qubits=[0],basis_gates=basis_gates_1, spam_gates=spam_gates_1)
gstexp5.set_analysis_options(fitter='linear_inversion_gst',rescale_tp=True, rescale_cp=True)
gstdata5 = gstexp5.run(backend_fake_P).block_for_results()
print(gstdata5.analysis_results("GST Experiment properties"))
print(gstdata5.analysis_results("gst estimation of X_Rot_90"))
print('extra:\n', gstdata5.analysis_results("gst estimation of X_Rot_90").extra)

DbAnalysisResultV1
- name: GST Experiment properties
- value: {'fitter': 'linear_inversion_gst', 'fitter_time': 0.7054839134216309, 'rescale_cp': True, 'rescale_tp': True, 'GST gates': ['Id', 'X_Rot_90', 'Y_Rot_90'], 'GST SPAM gates': {'F0': ('Id',), 'F1': ('X_Rot_90',), 'F2': ('Y_Rot_90',), 'F3': ('X_Rot_90', 'X_Rot_90')}}
- device_components: ['Q0']
- verified: False
DbAnalysisResultV1
- name: gst estimation of X_Rot_90
- value: Choi([[ 0.49926136+0.j        ,  0.0140139 +0.48456003j,
       -0.00447987+0.49227578j,  0.4851436 +0.00625403j],
      [ 0.0140139 -0.48456003j,  0.48686766+0.j        ,
        0.49531295+0.00131715j,  0.01639554-0.48842777j],
      [-0.00447987-0.49227578j,  0.49531295-0.00131715j,
        0.52223907+0.j        ,  0.0166967 -0.50120278j],
      [ 0.4851436 -0.00625403j,  0.01639554+0.48842777j,
        0.0166967 +0.50120278j,  0.4916319 +0.j        ]],
     input_dims=(2,), output_dims=(2,))
- extra: <1 items>
- device_components: ['Q0']
- verified: False

#### Example for providing an arbitrary fitter initial guess for MLE fitter:

In [17]:
fitter_initial_gateset = {
 'meas': np.array([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)]),
 'rho': DensityMatrix(np.array([[1,0],[0,0]])),
 'Id':PTM(IGate()),
 'H': PTM(HGate()),
 'Y': PTM(YGate()),
 'X_Rot_90': PTM(U2Gate(-np.pi / 2, np.pi / 2)),
}


gstexp6 = GateSetTomography(qubits=[0], basis_gates=basis_gates_3, only_basis_gates=True)
gstexp6.set_analysis_options(fitter_initial_guess=fitter_initial_gateset)
gstdata6 = gstexp6.run(backend_fake_P).block_for_results()
print(gstdata6.analysis_results("GST Experiment properties"))
print(gstdata6.analysis_results("gst estimation of H"))
print('extra:\n', gstdata6.analysis_results("gst estimation of H").extra)



DbAnalysisResultV1
- name: GST Experiment properties
- value: {'fitter': 'scipy_optimizer_MLE_gst', 'fitter_time': 4.665173530578613, 'fitter_initial_guess': {'meas': array([0.70710678, 0.        , 0.        , 0.70710678]), 'rho': array([[0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j],
       [0.70710678+0.j]]), 'Id': PTM([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
     [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
     [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
     [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]],
    input_dims=(2,), output_dims=(2,)), 'H': PTM([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
     [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j],
     [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
     [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j]],
    input_dims=(2,), output_dims=(2,)), 'Y': PTM([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
     [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
     [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
     [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]],
    input_dims=(2,), output_dims=(2,)), 'X_Rot_90': PTM([[ 1.000000e

## Running gate set tomography on 2-qubits

GST experiment can be run on an arbitrary number of qubits. However, as solving the highly nonlinear MLE optimization problem takes a long time which sharply increases as we increase the number of qubits, it is almost non possible to run GST on three qubits with MLE fitter.

To run GST on two qubits:

In [18]:
qstexp7 = GateSetTomography(qubits=[0,1], basis_gates='default', spam_gates='default')
qstexp7.set_analysis_options(fitter='scipy_optimizer_MLE_gst', fitter_initial_guess='default')
qstdata7 = qstexp7.run(backend_fake_P).block_for_results()
print(qstdata7.analysis_results("GST Experiment properties"))
print(qstdata7.analysis_results("gst estimation of CX"))
print('extra:\n', qstdata7.analysis_results("gst estimation of CX").extra)

DbAnalysisResultV1
- name: GST Experiment properties
- value: {'fitter': 'scipy_optimizer_MLE_gst', 'fitter_time': 6784.263683080673, 'fitter_initial_guess': 'linear_inversion', 'GST gates': ['I I', 'X I', 'I X', 'RZ_pi_over_3 I', 'I RZ_pi_over_3', 'I SX', 'SX I', 'CX'], 'GST SPAM gates': {'F0': ('I I',), 'F1': ('X I',), 'F2': ('I X',), 'F3': ('X I', 'I X'), 'F4': ('I X', 'SX I', 'CX'), 'F5': ('I SX', 'I RZ_pi_over_3', 'I SX'), 'F6': ('SX I', 'RZ_pi_over_3 I', 'SX I'), 'F7': ('X I', 'I SX', 'I RZ_pi_over_3', 'I SX'), 'F8': ('I X', 'I SX', 'CX', 'I SX'), 'F9': ('I X', 'SX I', 'RZ_pi_over_3 I', 'SX I'), 'F10': ('RZ_pi_over_3 I', 'RZ_pi_over_3 I', 'RZ_pi_over_3 I', 'SX I'), 'F11': ('I RZ_pi_over_3', 'I RZ_pi_over_3', 'I RZ_pi_over_3', 'I SX'), 'F12': ('I SX', 'SX I', 'CX', 'I SX'), 'F13': ('X I', 'I RZ_pi_over_3', 'I RZ_pi_over_3', 'I RZ_pi_over_3', 'I SX'), 'F14': ('I SX', 'I RZ_pi_over_3', 'SX I', 'CX', 'I SX'), 'F15': ('RZ_pi_over_3 I', 'RZ_pi_over_3 I', 'RZ_pi_over_3 I', 'CX', 'I SX',

## References

[1] Greenbaum, Daniel. "Introduction to quantum gate set tomography." arXiv preprint arXiv:1509.02921 (2015).

In [45]:
import qiskit.tools.jupyter
%qiskit_copyright